In [1]:
import os
import pandas as pd
from datetime import datetime, timezone
from binance.client import Client


def get_klines_df(symbol, interval):
    # create the Binance client, no need for api key
    client = Client("", "")

    # generate data folder
    if not os.path.exists("Data"):
        os.mkdir("Data")
    # check file exist
    file_name = f"Data//{symbol}_{interval}.csv"
    if os.path.exists(file_name):
        # read old file
        file_data = pd.read_csv(file_name)
        # file_data = file_data.astype("float")
        old_ts = file_data.iloc[-1][0]
        old_time_str = datetime.fromtimestamp(old_ts / 1000, timezone.utc).strftime(
            "%d %b %Y %H:%M:%S"
        )
        # get new data
        new_data = client.futures_historical_klines(
            symbol, interval, old_time_str)
        now_data_df = pd.DataFrame(new_data)
        now_data_df.columns = [str(i) for i in now_data_df.columns]

        # combind data
        if now_data_df.shape[0] > 0:
            dataframe_data_new = pd.concat([file_data, now_data_df], axis=0)
            dataframe_data_new = dataframe_data_new[
                ~dataframe_data_new["0"].duplicated(keep="last")
            ]
        else:
            dataframe_data_new = file_data.copy()
    else:
        # get new data
        start_str = datetime.strptime(
            "2020-01-01 00:00:00", "%Y-%m-%d %H:%M:%S"
        ).strftime("%d %b %Y %H:%M:%S")
        tmp_data = client.futures_historical_klines(
            symbol, interval, start_str)
        dataframe_data_new = pd.DataFrame(tmp_data)
    # columns naming
    dataframe_data_new.columns = [
        "Datetime",
        "Open",
        "High",
        "Low",
        "Close",
        "Volume",
        "Close_time",
        "Quote_asset_volume",
        "Number_of_trades",
        "Taker_buy_base_asset_volume",
        "Taker_buy_quote_asset_volume",
        "Ignore",
    ]
    # set index by datetime
    dataframe_data_new["Datetime"] = pd.to_datetime(
        dataframe_data_new["Datetime"] * 1000000, utc=True, unit="ns"
    )
    # dataframe_data_new = dataframe_data_new.astype("float")
    dataframe_data_new.to_csv(file_name, index=False)
    return dataframe_data_new

In [2]:
from binance.client import Client
import pandas as pd
from pandas_datareader import data
from datetime import datetime

symbol = "ETHBUSD"
interval = Client.KLINE_INTERVAL_30MINUTE
df = get_klines_df(symbol, interval)
print(df)

                       Datetime     Open     High      Low    Close   Volume  \
0     2021-06-16 03:30:00+00:00  2493.19  2578.98  2142.53  2522.43  187.229   
1     2021-06-16 04:00:00+00:00  2524.21  2524.89  2512.74  2516.95   66.383   
2     2021-06-16 04:30:00+00:00  2515.79  2537.51  2515.79  2529.40   60.795   
3     2021-06-16 05:00:00+00:00  2528.44  2532.59  2512.46  2515.41   37.792   
4     2021-06-16 05:30:00+00:00  2514.42  2525.78  2514.42  2521.82   42.431   
...                         ...      ...      ...      ...      ...      ...   
43976 2023-12-19 07:30:00+00:00  2211.00  2211.00  2211.00  2211.00        0   
43977 2023-12-19 08:00:00+00:00  2211.00  2211.00  2211.00  2211.00        0   
43978 2023-12-19 08:30:00+00:00  2211.00  2211.00  2211.00  2211.00        0   
43979 2023-12-19 09:00:00+00:00  2211.00  2211.00  2211.00  2211.00        0   
43980 2023-12-19 09:30:00+00:00  2211.00  2211.00  2211.00  2211.00        0   

          Close_time Quote_asset_volume

In [3]:
from backtesting import Backtest, Strategy #引入回測和交易策略功能

from backtesting.lib import crossover #從lib子模組引入判斷均線交會功能
from backtesting.test import SMA #從test子模組引入繪製均線功能

import pandas as pd #引入pandas讀取股價歷史資料CSV檔


class SmaCross(Strategy): #交易策略命名為SmaClass，使用backtesting.py的Strategy功能
    n1 = 5 #設定第一條均線日數為5日(周線)
    n2 = 20 #設定第二條均線日數為20日(月線)，這邊的日數可自由調整

    def init(self):
        self.sma1 = self.I(SMA, self.data.Close, self.n1) #定義第一條均線為sma1，使用backtesting.py的SMA功能算繪
        self.sma2 = self.I(SMA, self.data.Close, self.n2) #定義第二條均線為sma2，使用backtesting.py的SMA功能算繪

    def next(self):
        # if crossover(self.sma1, self.sma2): #如果周線衝上月線，表示近期是上漲的，則買入
        #     self.buy()
        # elif crossover(self.sma2, self.sma1): #如果周線再與月線交叉，表示開始下跌了，則賣出
        #     self.sell()
        if crossover(2300, self.data.Close):
            self.buy()
        elif crossover(self.data.Close, 2400):
            self.sell()

/Users/cheng/Library/Python/3.9/lib/python/site-packages/backtesting/_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

/Users/cheng/Library/Python/3.9/lib/python/site-packages/backtesting/test/__init__.py:8: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  return pd.read_csv(join(dirname(__file__), filename),
/Users/cheng/Library/Python/3.9/lib/python/site-packages/backtesting/test/__init__.py:8: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  return pd.read_csv(join(dirname(__file__), filename),


In [4]:
symbol = "ETHBUSD" #設定要測試的股票標的名稱
interval = Client.KLINE_INTERVAL_30MINUTE

df = pd.read_csv(f"./data/{symbol}_{interval}.csv", index_col=0) #pandas讀取資料，並將第1欄作為索引欄
df = df.interpolate() #CSV檔案中若有缺漏，會使用內插法自動補值，不一定需要的功能
df.index = pd.to_datetime(df.index) #將索引欄資料轉換成pandas的時間格式，backtesting才有辦法排序


In [5]:
test = Backtest(df, SmaCross, cash=10000, commission=.002)
# 指定回測程式為test，在Backtest函數中依序放入(資料來源、策略、現金、手續費)

result = test.run()
#執行回測程式並存到result中
print(result)

Start                     2021-06-16 03:30...
End                       2023-12-19 09:30...
Duration                    916 days 06:00:00
Exposure Time [%]                   99.936336
Equity Final [$]                   10742.5884
Equity Peak [$]                    15987.5484
Return [%]                           7.425884
Buy & Hold Return [%]              -12.346428
Return (Ann.) [%]                    2.384914
Volatility (Ann.) [%]              844.963357
Sharpe Ratio                         0.002823
Sortino Ratio                        0.022386
Calmar Ratio                         0.024192
Max. Drawdown [%]                  -98.583852
Avg. Drawdown [%]                   -5.039539
Max. Drawdown Duration      548 days 13:30:00
Avg. Drawdown Duration       21 days 18:45:00
# Trades                                    1
Win Rate [%]                            100.0
Best Trade [%]                       7.746117
Worst Trade [%]                      7.746117
Avg. Trade [%]                    

In [ ]:
print(result) # 直接print文字結果